<a href="https://colab.research.google.com/github/rkwi/master-thesis/blob/master/code/poisson_equation_neumann.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files

import platform, sys
python_version=platform.python_version()
from distutils.version import LooseVersion, StrictVersion

if ( LooseVersion(python_version) < LooseVersion("3.0.0")):
    print("Python3 is needed!");
    print("How to fix: Runtime/Change_runtime_type/Python 3");
    sys.exit()
try:
    from dolfin import *; from mshr import *
except ImportError as e:
    !apt-get install -y -qq software-properties-common python-software-properties module-init-tools
    !add-apt-repository -y ppa:fenics-packages/fenics
    !apt-get update -qq
    !apt install -y --no-install-recommends fenics
    from dolfin import *; from mshr import *
    
import matplotlib.pyplot as plt;
from IPython.display import clear_output, display; import time; import dolfin.common.plotting as fenicsplot 
import time
import numpy as np
import pandas as pd
from fenics import *
import os, sys, shutil

dolfin_version = dolfin.__version__
print ('dolfin version:', dolfin_version)

!rm -rf * # clean up all files
# Useful commands
# Remove an empty folder      : os.rmdir("my_results")
# Remove a folder with files  : shutil.rmtree("results")
# Make a folder               : os.mkdir("my_results")
# Runtime/Change_runtime_type/Python3

In [0]:
def errornorm_L1(u_e, u):
  V = u.function_space()
  mesh = V.mesh()
  degree = V.ufl_element().degree()
  W = FunctionSpace(mesh, 'P', degree + 4)
  u_e_W = interpolate(u_e, W)
  u_W = interpolate(u, W)
  e_W = Function(W)
  e_W.vector()[:] = u_e_W.vector().get_local() - u_W.vector().get_local()
  error = abs(e_W)*dx
  return abs(assemble(error))

def errornorm_Linf(u_e, u):
  S = FunctionSpace(u.function_space().mesh(),'P',u.function_space().ufl_element().degree() + 4)
  u_e_interpolated = interpolate(u_e,S)
  u_interpolated = interpolate(u,S)
  temp = u_e_interpolated.vector() - u_interpolated.vector()
  return norm(temp,'linf')

def create_error_table(N_list, L1_error, L2_error, Linf_error):
  row = len(L1_error)
  data_array = np.zeros((row,6))
  for i in range(row):
    for j in range(6):
      if j == 0:
        data_array[i][j] = L1_error[i]
      elif j == 1 and i < row - 1:
        data_array[i][j] = np.log2(L1_error[i]/L1_error[i+1])
      elif j == 2:
        data_array[i][j] = L2_error[i]
      elif j == 3 and i < row - 1:
        data_array[i][j] = np.log2(L2_error[i]/L2_error[i+1])
      elif j == 4:
        data_array[i][j] = Linf_error[i]
      elif j == 5 and i < row - 1:
        data_array[i][j] = np.log2(Linf_error[i]/Linf_error[i+1])
  table = pd.DataFrame(data_array)
  table.set_index([[str('1/' + str(N)) for N in N_list]],inplace=True)
  table.columns = [['L1','L1_rate','L2','L2_rate','Linf','Linf_rate']]
  return table

In [0]:
# Mixed Finite Element Method
from __future__ import print_function
from fenics import *
N_list = [16,32,64,128]
error_L2_list = []
u_list = []
u_exact_list = []
for N in N_list:
  interpolation_degree = 3
  mesh = UnitSquareMesh(N, N)
  V = FiniteElement("P",triangle,1)
  R = FiniteElement("R",triangle,0)
  W = FunctionSpace(mesh,V*R)

  u_exact = Expression('cos(pi*x[0])*cos(pi*x[1])', degree=interpolation_degree)

  (u, c) = TrialFunction(W)
  (v, d) = TestFunctions(W)
  f = Expression("2*pow(pi, 2)*cos(pi*x[0])*cos(pi*x[1])",degree=interpolation_degree)
  g = Constant(0)
  a = (inner(grad(u), grad(v)) + c*v + u*d)*dx
  L = f*v*dx + g*v*ds

  w = Function(W)
  solve(a == L, w)
  (u, temp) = w.split()
  u_list.append(u)
  u_exact_list.append(u_exact)

error_L2_list = []
error_L1_list = []
error_Linf_list = []
for i in range(len(u_list)):
  error_L2_list.append(errornorm(u_exact_list[i],u_list[i],'L2'))
  error_L1_list.append(errornorm_L1(u_exact_list[i],u_list[i]))
  error_Linf_list.append(errornorm_Linf(u_exact_list[i],u_list[i]))

In [0]:
u_table = create_error_table(N_list,error_L1_list,error_L2_list,error_Linf_list)

In [6]:
u_table


,L1,L1_rate,L2,L2_rate,Linf,Linf_rate
1/16,0.004397,1.987525,0.005339,1.985204,0.016615,1.765265
1/32,0.001109,1.996576,0.001348,1.995855,0.004888,1.802592
1/64,0.000278,1.999071,0.000338,1.998856,0.001401,1.827636
1/128,0.000070,0.000000,0.000085,0.000000,0.000395,0.000000


In [0]:
u_table.to_latex('u_table.txt',float_format='%.3E')

In [0]:
# Fix a Point Method
from __future__ import print_function
from fenics import *
N_list = [16,32,64,128]
error_L2_list = []
u_list = []
u_exact_list = []
for N in N_list:
  interpolation_degree = 3
  mesh = UnitSquareMesh(N, N)
  V = FunctionSpace(mesh, 'P', 1)

  u_exact = Expression('cos(pi*x[0])*cos(pi*x[1])', degree=interpolation_degree)

  u = TrialFunction(V)
  v = TestFunction(V)
  f = Expression('2*pow(pi,2)*cos(pi*x[0])*cos(pi*x[1])', degree=interpolation_degree)
  F = dot(grad(u),grad(v))*dx - f*v*dx
  a = lhs(F)
  L = rhs(F)
  
  bc = DirichletBC(V, 0, 'near(x[0],0.5) && near(x[1],0.5)', 'pointwise')
  u = Function(V)
  solve(a == L, u, bc)
  
  u_list.append(u)
  u_exact_list.append(u_exact)
  
error_L2_list = []
error_L1_list = []
error_Linf_list = []
for i in range(len(u_list)):
  error_L2_list.append(errornorm(u_exact_list[i],u_list[i],'L2'))
  error_L1_list.append(errornorm_L1(u_exact_list[i],u_list[i]))
  error_Linf_list.append(errornorm_Linf(u_exact_list[i],u_list[i]))

In [0]:
u_table = create_error_table(N_list,error_L1_list,error_L2_list,error_Linf_list)

In [9]:
u_table

,L1,L1_rate,L2,L2_rate,Linf,Linf_rate
1/16,0.004391,1.987605,0.005341,1.985262,0.016465,1.763006
1/32,0.001107,1.996631,0.001349,1.995870,0.004851,1.801134
1/64,0.000277,1.999089,0.000338,1.998860,0.001392,1.826563
1/128,0.000069,0.000000,0.000085,0.000000,0.000392,0.000000


In [0]:
u_table.to_latex('u_table.txt',float_format='%.3E')